In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import csv

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head() #datset visualization

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df = df.dropna() #dropping the nan values

In [ ]:
df.shape

(18285, 5)

In [ ]:
X = df.drop('label', axis = 1)

In [ ]:
y = df['label']

In [ ]:
X.shape #shape of input 

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.2.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

In [ ]:
messages = X.copy() #making copy of input

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [ ]:
#data cleaning
for i in range(len(messages)):
  review = re.sub('[^A-Za-z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
X.shape

(18285, 4)

In [ ]:
vocab_size = 5000
onehot_rep = [one_hot(word,vocab_size) for word in corpus]

In [ ]:
onehot_rep

[[3170, 4305, 2331, 364, 3027, 3894, 3628, 788, 4880, 3499],
 [2739, 643, 3225, 302, 548, 4831, 4661],
 [698, 3041, 1633, 1635],
 [2788, 2787, 2368, 2569, 3811, 4691],
 [4922, 548, 4300, 3044, 1878, 4840, 548, 1135, 536, 3945],
 [258,
  3729,
  4231,
  2509,
  4634,
  4096,
  2331,
  1384,
  1085,
  149,
  1904,
  4804,
  4039,
  726,
  4661],
 [3275, 3223, 78, 4528, 3991, 1759, 714, 3871, 1619, 4382, 4285],
 [2239, 4161, 4386, 3634, 1464, 60, 4096, 174, 1619, 4382, 4285],
 [882, 1670, 2315, 4844, 4604, 1167, 1499, 1258, 4096, 534],
 [965, 1691, 1761, 4371, 708, 1224, 1808, 4507],
 [154, 1683, 2291, 1075, 3141, 3900, 4808, 3791, 4009, 3598, 3585],
 [2569, 672, 3027, 1167, 4096, 1464],
 [744, 3541, 83, 2930, 1063, 2185, 154, 4862, 4032],
 [1227, 269, 3997, 3537, 4622, 23, 4958, 1619, 4382, 4285],
 [4156, 3605, 367, 476, 2205, 1619, 4382, 4285],
 [2685, 3549, 1099, 1677, 4366, 2801, 1717, 4495, 1494, 503],
 [1701, 3810, 643],
 [2991, 105, 949, 4346, 4096, 1957, 3064, 4661],
 [815, 245, 3

In [ ]:
#padding the sentences for the embedding layer
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre',maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ...  788 4880 3499]
 [   0    0    0 ...  548 4831 4661]
 [   0    0    0 ... 3041 1633 1635]
 ...
 [   0    0    0 ... 1619 4382 4285]
 [   0    0    0 ... 4149 3381 4833]
 [   0    0    0 ... 3281  982 4055]]


In [ ]:
len(embedded_docs)

18285

In [ ]:
embedded_docs[10]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,  154, 1683,
       2291, 1075, 3141, 3900, 4808, 3791, 4009, 3598, 3585], dtype=int32)

In [ ]:
#creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length= sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=0)

In [ ]:
#training the model
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 11s 48ms/step - loss: 0.2992 - accuracy: 0.8604 - val_loss: 0.1987 - val_accuracy: 0.9128
Epoch 2/10
229/229 [==============================] - 10s 45ms/step - loss: 0.1425 - accuracy: 0.9433 - val_loss: 0.1908 - val_accuracy: 0.9185
Epoch 3/10
229/229 [==============================] - 10s 44ms/step - loss: 0.1035 - accuracy: 0.9627 - val_loss: 0.2140 - val_accuracy: 0.9185
Epoch 4/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0755 - accuracy: 0.9742 - val_loss: 0.2194 - val_accuracy: 0.9185
Epoch 5/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0605 - accuracy: 0.9789 - val_loss: 0.2655 - val_accuracy: 0.9191
Epoch 6/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0366 - accuracy: 0.9896 - val_loss: 0.3426 - val_accuracy: 0.9171
Epoch 7/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0258 - accuracy: 0.9923 - val_loss: 0.3303 - val_accuracy:

In [ ]:
#prediction
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

array([[1891,  149],
       [ 176, 1441]])

In [ ]:
#accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9111293409898824

In [ ]:
#adding dropout layers
from tensorflow.keras.layers import Dropout

#creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length= sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#training the model
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 11s 48ms/step - loss: 0.2939 - accuracy: 0.8640 - val_loss: 0.1914 - val_accuracy: 0.9191
Epoch 2/10
229/229 [==============================] - 10s 45ms/step - loss: 0.1490 - accuracy: 0.9402 - val_loss: 0.1856 - val_accuracy: 0.9199
Epoch 3/10
229/229 [==============================] - 10s 45ms/step - loss: 0.1171 - accuracy: 0.9547 - val_loss: 0.2085 - val_accuracy: 0.9202
Epoch 4/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0965 - accuracy: 0.9648 - val_loss: 0.2321 - val_accuracy: 0.9147
Epoch 5/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0769 - accuracy: 0.9727 - val_loss: 0.2325 - val_accuracy: 0.9199
Epoch 6/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0569 - accuracy: 0.9812 - val_loss: 0.3017 - val_accuracy: 0.9174
Epoch 7/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0462 - accuracy: 0.9846 - val_loss: 0.3154 - val_accuracy:

In [ ]:
#predicting the output
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1869,  171],
       [ 138, 1479]])

In [ ]:
#accuracy calaculation
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9155045118949959